# Parse "Table of Nuclear Electric Quadrupole Moments"

source: https://www-nds.iaea.org/publications/indc/indc-nds-0833/

In [1]:
from fractions import Fraction
from pathlib import Path
import re

import camelot
from camelot.io import read_pdf
import pandas as pd
import numpy as np
import polars as pl

In [2]:
pl.Config.set_tbl_rows(100)

polars.config.Config

In [22]:
pdf = Path("../data/indc-nds-0833.pdf")
# tabs = read_pdf(str(pdf), pages="19-66", flavor="stream", edge_tol=500, r_tol=10)
tabs = read_pdf(str(pdf), pages="19-66", flavor="stream", columns=['115,171,212,261,306,385,444,508,585'], table_areas=['50,545,701,50'])

In [32]:
df = pd.concat([t.df.replace("", None).dropna(axis="index", how="all") for t in tabs])
# drop first row and set columns
df = df.iloc[1:]
df.columns = ["Element", "Isotope", "E(level)", "T1/2", "s/p", "Q(b)", "ref.std.", "method", "NSR Keynumber", "Journal Reference"]
# save
df.to_parquet("nuc-el-quad-mom-raw.parquet")

## Process extracted table

In [63]:
qmom = pl.read_parquet("nuc-el-quad-mom-raw.parquet").select(["Isotope", "E(level)", "s/p", "Q(b)"])

# drop all rows where all values are null and keep only ground state nuclei
qmom = qmom.drop_nulls(subset="Q(b)").filter(pl.col("E(level)") == "0")

# get atomic and mass number and symbol
pattern = r"(\d+)\s*([A-Za-z]+)\s*(\d+)"
qmom = qmom.with_columns(
    captures=pl.col("Isotope").str.extract_groups(pattern)
).with_columns(
    atomic_number=pl.col("captures").struct["1"].cast(pl.Int64),
    symbol=pl.col("captures").struct["2"],
    mass_number=pl.col("captures").struct["3"].cast(pl.Int64),
).drop("captures")

In [64]:
# parse spin and parity
spin_re = r"(?P<spin>\d+/\d+|\d+)(?P<parity>[+-]?)"
qmom = qmom.with_columns(captures=pl.col("s/p").str.extract_groups(spin_re)).unnest(
    "captures"
)

In [65]:
# parse nuclear electric quadrupole moments and uncertainties
val_unc_re = r"(?P<value_sign>[+-])?(?P<value_lead>\d+)\.?(?P<value_decimals>\d+)?\s*\(?(?P<value_unc>\d+(?:\.\d+)?)\)?"
qmom = (
    qmom.with_columns(captures=pl.col("Q(b)").str.extract_groups(val_unc_re))
    .unnest("captures")
    .with_columns(
        value_precision=pl.col("value_decimals").str.len_chars().cast(pl.Int32),
    )
    .with_columns(
        quadrupole_moment=pl.concat_str(
            pl.col("value_sign"),
            pl.col("value_lead"),
            pl.lit("."),
            pl.col("value_decimals"),
            ignore_nulls=True,
        ).cast(pl.Float32),
        quadrupole_moment_uncertainty=pl.lit(10.0, dtype=pl.Float32).pow(
            -pl.col("value_precision").cast(pl.Float32)
        )
        * pl.col("value_unc").cast(pl.Float32),
    )
)

In [66]:
qmom

Isotope,E(level),s/p,Q(b),atomic_number,symbol,mass_number,spin,parity,value_sign,value_lead,value_decimals,value_unc,value_precision,quadrupole_moment,quadrupole_moment_uncertainty
str,str,str,str,i64,str,i64,str,str,str,str,str,str,i32,f32,f32
"""1 H 2""","""0""","""1+""","""+0.0028578(3)""",1,"""H""",2,"""1""","""+""","""+""","""0""","""0028578""","""3""",7,0.002858,3.0000e-7
"""3 Li 6""","""0""","""1+""","""-0.000806(6)""",3,"""Li""",6,"""1""","""+""","""-""","""0""","""000806""","""6""",6,-0.000806,0.000006
"""3 Li 7""","""0""","""3/2-""","""-0.0400(3)""",3,"""Li""",7,"""3/2""","""-""","""-""","""0""","""0400""","""3""",4,-0.04,0.0003
"""3 Li 8""","""0""","""2+""","""+0.0314(2)""",3,"""Li""",8,"""2""","""+""","""+""","""0""","""0314""","""2""",4,0.0314,0.0002
"""3 Li 9""","""0""","""3/2-""","""-0.0304(2)""",3,"""Li""",9,"""3/2""","""-""","""-""","""0""","""0304""","""2""",4,-0.0304,0.0002
"""3 Li 11""","""0""","""3/2-""","""(-)0.0333(5)""",3,"""Li""",11,"""3/2""","""-""",null,"""0""","""0333""","""5""",4,0.0333,0.0005
"""4 Be 9""","""0""","""3/2-""","""+0.0529(4)""",4,"""Be""",9,"""3/2""","""-""","""+""","""0""","""0529""","""4""",4,0.0529,0.0004
"""5 B 8""","""0""","""2+""","""+0.0643(14)""",5,"""B""",8,"""2""","""+""","""+""","""0""","""0643""","""14""",4,0.0643,0.0014
"""5 B 10""","""0""","""3+""","""+0.0846(2)""",5,"""B""",10,"""3""","""+""","""+""","""0""","""0846""","""2""",4,0.0846,0.0002


## Fetch isotope table for comparison

In [52]:
from mendeleev.fetch import fetch_table

In [53]:
isotopes = pl.from_pandas(fetch_table("isotopes"))
isotopes

id,atomic_number,mass,abundance,mass_number,mass_uncertainty,is_radioactive,half_life,half_life_unit,spin,g_factor,quadrupole_moment,parity,discovery_year,g_factor_uncertainty,abundance_uncertainty,half_life_uncertainty,quadrupole_moment_uncertainty
i64,i64,f64,f64,i64,f64,i64,f64,str,str,f64,f64,str,f64,f64,f64,f64,f64
1,1,1.007825,99.9855,1,1.4000e-11,0,null,null,"""1/2""",5.585695,0.0,"""+""",1920.0,1.8000e-8,0.0078,null,0.0
2,1,2.014102,0.0145,2,1.5000e-11,0,null,null,"""1""",0.857438,null,"""+""",1932.0,5.0000e-9,0.0078,null,null
3,1,3.016049,null,3,8.0000e-11,1,12.32,"""year""","""1/2""",5.957925,0.0,"""+""",1934.0,2.8000e-8,null,0.02,0.0
4,2,3.016029,0.0002,3,6.0000e-11,0,null,null,"""1/2""",-4.25525,0.0,"""+""",1934.0,6.0000e-8,0.0002,null,0.0
5,3,3.030775,null,3,0.002147,1,null,null,"""3/2""",null,null,"""-""",null,null,null,null,null
6,1,4.026432,null,4,0.000107,1,139.0,"""ysec""","""2""",null,null,"""-""",1981.0,null,null,10.0,null
7,2,4.002603,99.9998,4,1.6000e-10,0,null,null,"""0""",0.0,0.0,"""+""",1908.0,0.0,0.0002,null,0.0
8,3,4.027186,null,4,0.000228,1,91.0,"""ysec""","""2""",null,null,"""-""",1965.0,null,null,9.0,null
9,1,5.035311,null,5,0.000096,1,86.0,"""ysec""","""1/2""",null,0.0,"""+""",1987.0,null,null,6.0,0.0


In [68]:
qmom.select(
    ['atomic_number', 'symbol', 'mass_number', 'spin', 'parity', 'quadrupole_moment', 'quadrupole_moment_uncertainty']
    ).join(isotopes, on=["atomic_number", "mass_number"], suffix="_true", how="left"
          ).select(
    ['atomic_number', 'symbol', 'mass_number',
     'spin', 'spin_true',
     'parity', 'parity_true',
     'quadrupole_moment', 'quadrupole_moment_true',
     'quadrupole_moment_uncertainty','quadrupole_moment_uncertainty_true']
    )

atomic_number,symbol,mass_number,spin,spin_true,parity,parity_true,quadrupole_moment,quadrupole_moment_true,quadrupole_moment_uncertainty,quadrupole_moment_uncertainty_true
i64,str,i64,str,str,str,str,f32,f64,f32,f64
1,"""H""",2,"""1""","""1""","""+""","""+""",0.002858,null,3.0000e-7,null
3,"""Li""",6,"""1""","""1""","""+""","""+""",-0.000806,-0.000806,0.000006,0.000006
3,"""Li""",7,"""3/2""","""3/2""","""-""","""-""",-0.04,-0.04,0.0003,0.0003
3,"""Li""",8,"""2""","""2""","""+""","""+""",0.0314,0.0314,0.0002,0.0002
3,"""Li""",9,"""3/2""","""3/2""","""-""","""-""",-0.0304,-0.0304,0.0002,0.0002
3,"""Li""",11,"""3/2""","""3/2""","""-""","""-""",0.0333,-0.0333,0.0005,0.0005
4,"""Be""",9,"""3/2""","""3/2""","""-""","""-""",0.0529,0.0529,0.0004,0.0004
5,"""B""",8,"""2""","""2""","""+""","""+""",0.0643,0.0643,0.0014,0.0014
5,"""B""",10,"""3""","""3""","""+""","""+""",0.0846,0.0846,0.0002,0.0002


## Update values in mendleev db

In [61]:
from mendeleev.db import get_session, get_engine
from mendeleev.models import Isotope

In [62]:
session = get_session(read_only=False)

In [70]:
fields = {"spin", "parity", "quadrupole_moment", "quadrupole_moment_uncertainty"}
for row in qmom.iter_rows(named=True):
    iso = (
        session.query(Isotope)
        .filter_by(atomic_number=row["atomic_number"], mass_number=row["mass_number"])
        .update({k: v for k, v in row.items() if k in fields})
    )
    # session.rollback()
    session.commit()
session.close()

## Validate

In [71]:
from mendeleev import H

In [73]:
H.isotope(2).quadrupole_moment

0.002857800107449293